In [ ]:
!pip install unzip
!wget https://nlp.stanford.edu/data/glove.6B.zip
!unzip /content/glove.6B.zip

In [11]:
import pandas as pd
import numpy as np
import gc
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.optim import Adam

import gensim
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.metrics import classification_report, accuracy_score


In [12]:
import os

# Set CUDA_VISIBLE_DEVICES environment variable
os.environ['CUDA_VISIBLE_DEVICES'] = '1'  # Replace '0' with the GPU IDs you want to use

# You can verify that it's set by printing it
print(os.environ['CUDA_VISIBLE_DEVICES'])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

1


In [13]:
# TODO: use wget
glovefile = 'glove.6B.100d.txt'
glove_file_datapath = datapath(glovefile)
tmp_file = get_tmpfile('word2vec.txt')

_ = glove2word2vec(glovefile, tmp_file)
word2vec_weights = gensim.models.KeyedVectors.load_word2vec_format(tmp_file)
print(word2vec_weights['cat'])

/tmp/ipykernel_126737/627634055.py:6: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  _ = glove2word2vec(glovefile, tmp_file)


[ 0.23088    0.28283    0.6318    -0.59411   -0.58599    0.63255
  0.24402   -0.14108    0.060815  -0.7898    -0.29102    0.14287
  0.72274    0.20428    0.1407     0.98757    0.52533    0.097456
  0.8822     0.51221    0.40204    0.21169   -0.013109  -0.71616
  0.55387    1.1452    -0.88044   -0.50216   -0.22814    0.023885
  0.1072     0.083739   0.55015    0.58479    0.75816    0.45706
 -0.28001    0.25225    0.68965   -0.60972    0.19578    0.044209
 -0.31136   -0.68826   -0.22721    0.46185   -0.77162    0.10208
  0.55636    0.067417  -0.57207    0.23735    0.4717     0.82765
 -0.29263   -1.3422    -0.099277   0.28139    0.41604    0.10583
  0.62203    0.89496   -0.23446    0.51349    0.99379    1.1846
 -0.16364    0.20653    0.73854    0.24059   -0.96473    0.13481
 -0.0072484  0.33016   -0.12365    0.27191   -0.40951    0.021909
 -0.6069     0.40755    0.19566   -0.41802    0.18636   -0.032652
 -0.78571   -0.13847    0.044007  -0.084423   0.04911    0.24104
  0.45273   -0.18682 

In [29]:
#@title Load data

train_url = 'https://raw.githubusercontent.com/deckerkrogh/nlp243_data/main/datasets/task3_train.json'
train_df = pd.read_json(train_url)
val_url = 'https://raw.githubusercontent.com/deckerkrogh/nlp243_data/main/datasets/task3_dev.json'
val_df = pd.read_json(val_url)
test_url = 'https://raw.githubusercontent.com/deckerkrogh/nlp243_data/main/datasets/task3_test.json'
test_df = pd.read_json(test_url)

In [30]:
#@title Sequencer

class PreTrainedSequencer(object):
    # Maps text to index and then to corresponding embedding
    # Source: Jesh
    def __init__(self, corpus, gensim_w2v, embedding_dim, bos_token='<s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>'):

        print(corpus)
        self.idx2word = {}
        self.word2idx = {}
        self.embedding_dim = embedding_dim
        self.w2v = gensim_w2v

        self.w2v.add_vectors([bos_token], [np.random.uniform(low=-1, high=1.0, size=(self.embedding_dim,))])
        self.w2v.add_vectors([eos_token], [np.random.uniform(low=-1, high=1.0, size=(self.embedding_dim,))])
        self.w2v.add_vectors([unk_token], [np.random.uniform(low=-1, high=1.0, size=(self.embedding_dim,))])
        self.w2v.add_vectors([pad_token], [np.random.uniform(low=-1, high=1.0, size=(self.embedding_dim,))])

        self.bos_index = self.add_token(bos_token)
        self.eos_index = self.add_token(eos_token)
        self.unk_index = self.add_token(unk_token)
        self.pad_index = self.add_token(pad_token)
        self.tokenizer = lambda text: [t for t in text.split(' ')]

        for _sentence in corpus:
            #print(_sentence)
            for _token in self.tokenizer(_sentence):
                self.add_token(_token)
        self.pre_trained_embeddings = torch.zeros([len(self.idx2word.keys()), self.embedding_dim])

        for idx, word in self.idx2word.items():
            if self.w2v.has_index_for(word):
                self.pre_trained_embeddings[idx] = torch.tensor(self.w2v.get_vector(self.w2v.key_to_index.get(word), norm=True))
            else:
                self.pre_trained_embeddings[idx] = torch.tensor(self.w2v.get_vector(self.w2v.key_to_index.get(unk_token), norm=True))

    def add_token(self, token):
        if token not in self.word2idx:
            self.word2idx[token] = new_index = len(self.word2idx)
            self.idx2word[new_index] = token
            return new_index
        else:
            return self.word2idx[token]

    def encode(self, text):
        tokens = self.tokenizer(text)
        sequence = []
        sequence.append(self.bos_index)

        for token in tokens:

            index = self.word2idx.get(token, self.unk_index)
            sequence.append(index)

        sequence.append(self.eos_index)
        return sequence

    def create_padded_tensor(self, sequences):
        lengths = [len(sequence) for sequence in sequences]
        max_seq_len = max(lengths)
        tensor = torch.full((len(sequences), max_seq_len), self.pad_index, dtype=torch.long)

        for i, sequence in enumerate(sequences):
            for j, token in enumerate(sequence):
                tensor[i][j] = token

        return tensor, lengths

In [31]:
#@title Dataset class

class MELDDataset(Dataset):
    def __init__(self, df, sequencer, emotion_to_label):
        self.sequencer = sequencer
        self.emotion_to_label = emotion_to_label
        self.utt_pairs = []

        for i, conversation in df.iterrows():
            target = conversation['utterances'][-1]
            target_emotion = self.emotion_to_label[conversation['emotions'][-1]]
            for utt, trigger, emotion in zip(conversation['utterances'], conversation['triggers'], conversation['emotions']):
                self.utt_pairs.append([target, utt, trigger, target_emotion, self.emotion_to_label[emotion]])

    def __getitem__(self, index):
        utt_pair = self.utt_pairs[index]
        x_target = self.sequencer.encode(utt_pair[0])
        x_candidate = self.sequencer.encode(utt_pair[1])
        y = utt_pair[2]
        target_emotion = utt_pair[3]
        candidate_emotion = utt_pair[4]
        return x_target, x_candidate, y, target_emotion, candidate_emotion

    def __len__(self):
        return len(self.utt_pairs)


In [32]:
#@title Model

class DualLSTMWithEmotion(nn.Module):
    def __init__(self, pad_index, embedding_dim, hidden_size, num_emotions, num_layers=2, dropout_p=0.5, pre_trained_embeddings=None, tunable_pre_trained_embedding=False):
        super(DualLSTMWithEmotion, self).__init__()

        self.pad_index = pad_index
        self.pre_trained_embeddings = pre_trained_embeddings
        self.tunable_pre_trained_embedding = tunable_pre_trained_embedding if pre_trained_embeddings is not None else False

        # Embedding layer
        if pre_trained_embeddings is not None:
            print("Using pre-trained embeddings")
            if tunable_pre_trained_embedding:
                self.tunable_embedding = nn.Embedding.from_pretrained(pre_trained_embeddings, freeze=False)
            self.embedding = nn.Embedding.from_pretrained(pre_trained_embeddings, freeze=True)
        else:
            self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # LSTM layers
        self.target_lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout_p if num_layers > 1 else 0,
            bidirectional=False,
            batch_first=True
        )
        self.cand_lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout_p if num_layers > 1 else 0,
            bidirectional=False,
            batch_first=True
        )

        # Output layer for trigger classification
        self.trigger_classifier = nn.Linear(hidden_size * 3, 1)  # Adjusted size

        # Output layers for emotion classification
        self.target_emotion_classifier = nn.Linear(hidden_size, num_emotions)
        self.candidate_emotion_classifier = nn.Linear(hidden_size, num_emotions)

        # Additional layer for processing the difference in emotions (if needed)
        self.emotion_diff_layer = nn.Linear(num_emotions, hidden_size)

    def forward(self, targets, target_l, candidates, cand_l):
        # Process 'targets' and 'candidates'
        embed_target = self.embedding(targets)
        embed_cand = self.embedding(candidates)

        # Verify the shape of the embedding output
       # print(f"Embedding target shape: {embed_target.shape}")  # Should be [batch_size, seq_len, embedding_dim]
       # print(f"Embedding candidate shape: {embed_cand.shape}")

        _, (target_h_n, _) = self.target_lstm(embed_target)
        _, (cand_h_n, _) = self.cand_lstm(embed_cand)

        #print(f"LSTM target output shape: {target_h_n.shape}")
        #print(f"LSTM candidate output shape: {cand_h_n.shape}")

        # Emotion classification for target and candidate
        target_emotion_logits = self.target_emotion_classifier(target_h_n[-1])
        candidate_emotion_logits = self.candidate_emotion_classifier(cand_h_n[-1])

        # Process the difference in emotions
        emotion_diff = self.emotion_diff_layer(torch.abs(target_emotion_logits - candidate_emotion_logits))

        # Trigger prediction using concatenated context including emotion difference
        context_vector = torch.cat((target_h_n[-1], cand_h_n[-1], emotion_diff), dim=1)
        trigger_logits = self.trigger_classifier(context_vector)

        return trigger_logits, target_emotion_logits, candidate_emotion_logits

In [33]:
def prepare_batch(batch, sequencer):
    # Unpack the batch
    target_utts, candidate_utts, triggers, target_emotions, candidate_emotions = zip(*batch)

    # Create padded tensors for target and candidate utterances
    input_target_tensor, target_lengths = sequencer.create_padded_tensor(target_utts)
    input_candidate_tensor, cand_lengths = sequencer.create_padded_tensor(candidate_utts)

    # Convert triggers and emotions to tensors
    triggers_tensor = torch.tensor(triggers, dtype=torch.float32)
    target_emotions_tensor = torch.tensor(target_emotions, dtype=torch.long)
    candidate_emotions_tensor = torch.tensor(candidate_emotions, dtype=torch.long)

    return input_target_tensor, target_lengths, input_candidate_tensor, cand_lengths, triggers_tensor, target_emotions_tensor, candidate_emotions_tensor

In [34]:
from collections import Counter

emotion_to_label = {"anger": 0, "disgust": 1, "fear": 2, "joy": 3, "neutral": 4, "sadness": 5, "surprise": 6}
label_to_emotion = {label: emotion for emotion, label in emotion_to_label.items()}

# Initialize an empty DataFrame
emotion_df = pd.DataFrame(columns=['utterance', 'emotions'])

# Initialize a Counter object to count emotions
emotion_counts = Counter()

for i, episode in train_df.iterrows():
    # Count emotions in this episode
    episode_emotion_counts = Counter(episode['emotions'])
    emotion_counts.update(episode_emotion_counts)

    # Create a DataFrame for this episode
    episode_df = pd.DataFrame({
        'utterance': episode['utterances'],
        'emotions': episode['emotions']
    })

    # Concatenate with the main DataFrame
    emotion_df = pd.concat([emotion_df, episode_df], ignore_index=True)

# Print the counts of each emotion
print(emotion_counts)

total_count = sum(emotion_counts.values())
class_weights = {emotion: total_count / count for emotion, count in emotion_counts.items()}
#print(class_weights)

# Convert class weights to align with numeric labels
class_weights_numeric = {emotion_to_label[emotion]: weight for emotion, weight in class_weights.items()}
print(class_weights_numeric)

# Ensure the order of weights matches the numeric labels
weights_tensor = torch.tensor([class_weights_numeric[i] for i in range(len(emotion_to_label))])
#print(weights_tensor)
weights_tensor = weights_tensor.to(device)

# Use these weights in your loss function
emotion_loss_function = torch.nn.CrossEntropyLoss(weight=weights_tensor)


Counter({'neutral': 12165, 'joy': 5018, 'surprise': 3644, 'anger': 3194, 'sadness': 2157, 'fear': 865, 'disgust': 815})
{3: 5.5516141889198884, 0: 8.721978710081403, 4: 2.2900123304562268, 5: 12.915159944367177, 6: 7.644895718990121, 2: 32.20578034682081, 1: 34.18159509202454}


In [35]:
# Parameters
embedding_dim = 100
batch_size = 32
hidden_size = 512
n_epochs = 5
num_emotions = 7
learning_rate = 0.001
trigger_loss_function = nn.BCEWithLogitsLoss()
emotion_loss_function = nn.CrossEntropyLoss()

train_utts = [utt for sublist in train_df['utterances'] for utt in sublist]
val_utts = [utt for sublist in val_df['utterances'] for utt in sublist]
test_utts = [utt for sublist in test_df['utterances'] for utt in sublist]
sequencer = PreTrainedSequencer(train_utts, word2vec_weights, embedding_dim)

model = DualLSTMWithEmotion(pad_index=sequencer.pad_index,
                            embedding_dim=embedding_dim,
                            hidden_size=hidden_size,
                            num_emotions=num_emotions,
                            num_layers=1,
                            dropout_p=0.1,
                            pre_trained_embeddings=sequencer.pre_trained_embeddings,
                            tunable_pre_trained_embedding=True)
model.to(device)
#model.target_lstm.flatten_parameters()
#model.cand_lstm.flatten_parameters()
print(model.to(device))

#  def __init__(self, embedding_dim, hidden_size, num_emotions, num_layers=1, dropout_p=0.1, pre_trained_embeddings=None):

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

train_dataset = MELDDataset(train_df, sequencer, emotion_to_label)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, collate_fn=lambda batch: prepare_batch(batch, sequencer))

val_dataset = MELDDataset(val_df, sequencer, emotion_to_label)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, collate_fn=lambda batch: prepare_batch(batch, sequencer))

test_dataset = MELDDataset(test_df, sequencer, emotion_to_label)
test_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, collate_fn=lambda batch: prepare_batch(batch, sequencer))

['Everything on your application looks really good\x97Ohh!', 'Just one last question umm, are you and your friends gonna be over here all the time like partying and hanging out?', "Oh don't worry, I'm not really a party girl.", "Whoa!! Now look, don't be just blurtin' stuff out. I want you to really think about your answers. Okay?", "You can call off the roommate search!  Hi! I'll be living here.", "Oh don't listen to him, he's just some guy who really wants the apartment, but I don't think he's gonna get it.", 'Relax every muscle in your body.', 'Listen to the plinky-plunky music.', 'Okay, now close you eyes, and think of a happy place.', 'Okay, tell me your happy place.', "Richard's living room, drinking wine.", 'All right. No, no, no, not a Richard thing, just put down the glass. And get out!', "I'm sorry, but that's my happy place.", 'Well, okay, fine, use my happy place.', "Okay, I'm just gonna, I have to ask that you don't move anything.", "All right, I'll try not to.", 'Okay, al

In [42]:
def train(model, train_loader, trigger_loss_function, target_emotion_loss_function, candidate_emotion_loss_function, optimizer, device, num_epochs=10):
    # Linearly increasing weight for candidate emotion loss
    lambda_trigger = lambda epoch: epoch / num_epochs
    # Decrease weight for target emotion loss after half the epochs
    lambda_target_emotion = lambda epoch: epoch / num_epochs
    # Increase weight for candidate emotion loss after half the epochs
    lambda_candidate_emotion = lambda_candidate_emotion = lambda epoch: 0.75 if epoch < num_epochs / 2 else 1
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        total_trigger_loss = 0
        total_target_emotion_loss = 0
        total_candidate_emotion_loss = 0

        for batch_idx, (targets, target_l, candidates, cand_l, trigger_labels, target_emotion_labels, candidate_emotion_labels) in enumerate(train_loader):
            # Move data to device
            targets, candidates = targets.to(device), candidates.to(device)
            trigger_labels, target_emotion_labels, candidate_emotion_labels = trigger_labels.to(device), target_emotion_labels.to(device), candidate_emotion_labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            trigger_logits, target_emotion_logits, candidate_emotion_logits = model(targets, target_l, candidates, cand_l)
            trigger_logits = trigger_logits.squeeze(-1)

            # Compute losses for all tasks
            trigger_loss = trigger_loss_function(trigger_logits, trigger_labels)
            target_emotion_loss = target_emotion_loss_function(target_emotion_logits, target_emotion_labels)
            candidate_emotion_loss = candidate_emotion_loss_function(candidate_emotion_logits, candidate_emotion_labels)

            # Apply lambda functions to each loss
            weighted_trigger_loss = lambda_trigger(epoch) * trigger_loss
            weighted_target_emotion_loss = lambda_target_emotion(epoch) * target_emotion_loss
            weighted_candidate_emotion_loss = lambda_candidate_emotion(epoch) * candidate_emotion_loss

            # Combine losses with dynamic weights
            loss = weighted_trigger_loss + weighted_target_emotion_loss + weighted_candidate_emotion_loss
            total_loss += loss.item()
            total_trigger_loss += weighted_trigger_loss.item()
            total_target_emotion_loss += weighted_target_emotion_loss.item()
            total_candidate_emotion_loss += weighted_candidate_emotion_loss.item()

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            # Logging
            #if batch_idx % 100 == 0:
            #    print(f'Epoch: {epoch}, Batch: {batch_idx}, Total Loss: {loss.item()}, Trigger Loss: {trigger_loss.item()}, Target Emotion Loss: {target_emotion_loss.item()}, Candidate Emotion Loss: {candidate_emotion_loss.item()}')

        # Log epoch-level metrics
        avg_loss = total_loss / len(train_loader)
        avg_trigger_loss = total_trigger_loss / len(train_loader)
        avg_target_emotion_loss = total_target_emotion_loss / len(train_loader)
        avg_candidate_emotion_loss = total_candidate_emotion_loss / len(train_loader)
        print(f'End of Epoch {epoch}, Average Loss: {avg_loss}, Average Trigger Loss: {avg_trigger_loss}, Average Target Emotion Loss: {avg_target_emotion_loss}, Average Candidate Emotion Loss: {avg_candidate_emotion_loss}')


In [54]:
def run_training(model, optimizer, train_loader, valid_loader=None, n_epochs=n_epochs, device=device):
    train_running_losses = []

    # Define loss functions
    trigger_loss_function = nn.BCEWithLogitsLoss()
    target_emotion_loss_function = nn.CrossEntropyLoss()
    candidate_emotion_loss_function = nn.CrossEntropyLoss()  # Separate emotion loss function
    
    for epoch in range(n_epochs):
        print(f"EPOCH {epoch}")

        # Training step
        mean_running_loss = train(model, train_loader, trigger_loss_function, target_emotion_loss_function, candidate_emotion_loss_function, optimizer, device, num_epochs=1)
        train_running_losses.append(mean_running_loss)

    return train_running_losses


In [56]:
run_training(model, optimizer, train_loader, valid_loader=val_loader, n_epochs=15, device=device)

EPOCH 0
End of Epoch 0, Average Loss: 1.6811785219252178, Average Trigger Loss: 0.0, Average Target Emotion Loss: 1.3473620949704392, Average Candidate Emotion Loss: 0.33381642533806244
EPOCH 1
End of Epoch 0, Average Loss: 1.6209014290099029, Average Trigger Loss: 0.0, Average Target Emotion Loss: 1.285945851188987, Average Candidate Emotion Loss: 0.33495558031442246
EPOCH 2
End of Epoch 0, Average Loss: 1.594296069287542, Average Trigger Loss: 0.0, Average Target Emotion Loss: 1.2602236607378234, Average Candidate Emotion Loss: 0.33407240756600237
EPOCH 3
End of Epoch 0, Average Loss: 1.6072880442668047, Average Trigger Loss: 0.0, Average Target Emotion Loss: 1.2685795200028458, Average Candidate Emotion Loss: 0.3387085222195402
EPOCH 4
End of Epoch 0, Average Loss: 1.5813022470604265, Average Trigger Loss: 0.0, Average Target Emotion Loss: 1.2512053415547078, Average Candidate Emotion Loss: 0.3300969049582594
EPOCH 5
End of Epoch 0, Average Loss: 1.5310040307407675, Average Trigger 

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [55]:
# clear gpu cache
gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()

In [57]:
from sklearn.metrics import classification_report, accuracy_score

def evaluate(model, loader, device, label_to_emotion):
    model.eval()  # Set the model to evaluation mode
    all_trigger_preds = []
    all_trigger_true = []
    all_target_emotion_preds = []
    all_target_emotion_true = []
    all_candidate_emotion_preds = []
    all_candidate_emotion_true = []

    with torch.no_grad():  # Disable gradient calculation
        for batch in loader:
            targets, target_l, candidates, cand_l, triggers, target_emotions, candidate_emotions = [item.to(device) if isinstance(item, torch.Tensor) else item for item in batch]

            trigger_logits, target_emotion_logits, candidate_emotion_logits = model(targets, target_l, candidates, cand_l)

            # Trigger predictions
            trigger_probabilities = torch.sigmoid(trigger_logits)
            trigger_predictions = (trigger_probabilities > 0.3).float()  # Apply a threshold of 0.5
            all_trigger_preds.extend(trigger_predictions.cpu().numpy().flatten())
            all_trigger_true.extend(triggers.cpu().numpy().flatten())

            # Emotion predictions for target and candidate
            target_emotion_predictions = torch.argmax(target_emotion_logits, dim=1)
            candidate_emotion_predictions = torch.argmax(candidate_emotion_logits, dim=1)
            
            # Decode emotion labels
            decoded_target_emotions_preds = [label_to_emotion[label] for label in target_emotion_predictions.cpu().numpy()]
            decoded_candidate_emotions_preds = [label_to_emotion[label] for label in candidate_emotion_predictions.cpu().numpy()]
            decoded_target_emotions_true = [label_to_emotion[label] for label in target_emotions.cpu().numpy()]
            decoded_candidate_emotions_true = [label_to_emotion[label] for label in candidate_emotions.cpu().numpy()]

            all_target_emotion_preds.extend(decoded_target_emotions_preds)
            all_candidate_emotion_preds.extend(decoded_candidate_emotions_preds)
            all_target_emotion_true.extend(decoded_target_emotions_true)
            all_candidate_emotion_true.extend(decoded_candidate_emotions_true)

    # Classification Report for Trigger Prediction
    trigger_report = classification_report(all_trigger_true, all_trigger_preds, labels=[0, 1], output_dict=False)
    print("Trigger Classification Report:\n", trigger_report)

    # Classification Report for Target Emotion Prediction
    target_emotion_report = classification_report(all_target_emotion_true, all_target_emotion_preds)
    print("Target Emotion Classification Report:\n", target_emotion_report)

    # Classification Report for Candidate Emotion Prediction
    candidate_emotion_report = classification_report(all_candidate_emotion_true, all_candidate_emotion_preds)
    print("Candidate Emotion Classification Report:\n", candidate_emotion_report)

    # Calculate average accuracy for trigger task
    trigger_accuracy = accuracy_score(all_trigger_true, all_trigger_preds)
    print(f"Trigger Prediction Average Accuracy: {trigger_accuracy}")

In [58]:
evaluate(model, train_loader, device, label_to_emotion)

Trigger Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.29      0.43     23402
           1       0.16      0.71      0.26      4456

    accuracy                           0.35     27858
   macro avg       0.50      0.50      0.34     27858
weighted avg       0.73      0.35      0.40     27858

Target Emotion Classification Report:
               precision    recall  f1-score   support

       anger       0.66      0.57      0.61      3844
     disgust       0.80      0.54      0.65      1215
        fear       0.61      0.54      0.58      1185
         joy       0.63      0.57      0.60      5169
     neutral       0.58      0.83      0.68      9493
     sadness       0.75      0.60      0.66      2677
    surprise       0.66      0.33      0.44      4275

    accuracy                           0.62     27858
   macro avg       0.67      0.57      0.60     27858
weighted avg       0.64      0.62      0.61     27858

Candi

In [59]:
evaluate(model, val_loader, device, label_to_emotion)

Trigger Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.28      0.42      2966
           1       0.15      0.71      0.25       522

    accuracy                           0.34      3488
   macro avg       0.50      0.50      0.33      3488
weighted avg       0.74      0.34      0.39      3488

Target Emotion Classification Report:
               precision    recall  f1-score   support

       anger       0.08      0.07      0.08       569
     disgust       0.05      0.05      0.05       126
        fear       0.10      0.10      0.10       156
         joy       0.25      0.23      0.24       708
     neutral       0.32      0.45      0.38      1082
     sadness       0.11      0.10      0.11       268
    surprise       0.17      0.09      0.12       579

    accuracy                           0.23      3488
   macro avg       0.16      0.16      0.15      3488
weighted avg       0.21      0.23      0.21      3488

Candi